In [6]:
import os

path = '../data'
try: 
    os.mkdir(path) 
except OSError as error: 
    print(error)

In [7]:
import requests
url = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
target_path = '../data/aclImdb_v1.tar.gz'

response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(target_path, 'wb') as f:
        f.write(response.raw.read())

In [8]:
import tarfile, zipfile
with tarfile.open('../data/aclImdb_v1.tar.gz') as tar:
    tar.extractall(path='../data')

In [14]:
import glob
data = {}
labels = {}
data_dir='../data/aclImdb'

for data_type in ['train', 'test']:
    data[data_type] = {}
    labels[data_type] = {}
    

    for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            print(data, '--------', labels)
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            print(path)
            files = glob.glob(path)
            #print(files,'\n')
            
            for f in files:
                with open(f, encoding="utf8") as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)

            
#print(data, '\n', labels0)

{'train': {'pos': []}} -------- {'train': {'pos': []}}
../data/aclImdb\train\pos\*.txt
{'train': {'pos': [], 'neg': []}} -------- {'train': {'pos': [], 'neg': []}}
../data/aclImdb\train\neg\*.txt
{'train': {'pos': [], 'neg': []}, 'test': {'pos': []}} -------- {'train': {'pos': [], 'neg': []}, 'test': {'pos': []}}
../data/aclImdb\test\pos\*.txt
{'train': {'pos': [], 'neg': []}, 'test': {'pos': [], 'neg': []}} -------- {'train': {'pos': [], 'neg': []}, 'test': {'pos': [], 'neg': []}}
../data/aclImdb\test\neg\*.txt


In [4]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [5]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


In [8]:
train_X[100]

"A low budget effort from Texas that's at least filmed well, but that is little consolation. Bad acting, or, should I say, bad over-acting, a pretty limp story line that's nothing new, bad special effects, bad, bad, bad. Seems like a bunch of young folks are putting together a haunted house for Halloween, which is done every year, but this year things are different. Has a long extended lesbian theme that is not only annoying but definitely fills out the empty spots, of which there are a lot. Putrid, puerile, definitely avoidable, at all costs."

In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
import re
from bs4 import BeautifulSoup

In [20]:
nltk.download("stopwords", quiet=True)
stemmer = PorterStemmer()
text = BeautifulSoup(train_X[100], "html.parser").get_text()
text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
words = text.split()
words = [w for w in words if w not in stopwords.words("english")]
print(len(words))
words =[PorterStemmer().stem(w) for w in words]

57


In [21]:
print(len(words))

57
